## Imports

In [1]:
import zmq
import zlib
import pyulog
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

## Socket config

In [2]:
port = '5555'
context = zmq.Context()
socket = context.socket(zmq.REQ)
socket.connect("tcp://0.0.0.0:%s" % port)

def send_zipped_pickle(socket, obj, flags=0, protocol=-1):
    p = pickle.dumps(obj, protocol)
    z = zlib.compress(p)
    return socket.send(z, flags=flags)

## Parser

In [3]:
ulg = None
def parse_file(filename):
    global ulg
    ulg = pyulog.ULog(filename)

    datadict = {}
    modified_data = {}
    for data in ulg.data_list:
        if data.multi_id > 0:
            name = f"{data.name}_{data.multi_id}"
        else:
            name = data.name
        datadict[name] = pd.DataFrame(data.data)
        datadict[name]['timestamp_tiplot'] = datadict[name]['timestamp'] /1e6
    return datadict

In [4]:
datadict = parse_file('/home/hamza/Documents/tiplot/logs/mega.ulg')

## Adding logged_messages to datadict

In [5]:
message_data = []
for message in ulg.logged_messages:
    message_dict = {}
    message_dict['timestamp'] = message.timestamp
    message_dict['message'] = message.message
    message_dict['timestamp_tiplot'] = message_dict['timestamp'] / 1e6
    message_data.append(message_dict)

datadict['ulg'] = {}
datadict['ulg'] = pd.DataFrame(message_data)

## Entities

In [6]:
  uav = {
    "name": "entity",
    "alpha": 1,
    "useRPY": False,
    "useXYZ": True,
    "pathColor": "#007bff",
    "wireframe": False,
    "color": "#ffffff",
    "tracked": True,
    "scale": 0.5,
    "position": {
      "table": "vehicle_local_position",
      "x": "x",
      "y": "y",
      "z": "z"
    },
    "attitude": {
      "table": "vehicle_attitude",
      "q0": "q[0]",
      "q1": "q[1]",
      "q2": "q[2]",
      "q3": "q[3]"
    }
  }

  uav_setpoint = {
    "name": "entity setpoint",
    "alpha": 1,
    "useRPY": False,
    "useXYZ": True,
    "pathColor": "#007bff",
    "wireframe": False,
    "color": "#ffffff",
    "tracked": True,
    "scale": 0.5,
    "position": {
      "table": "vehicle_local_position_setpoint",
      "x": "x",
      "y": "y",
      "z": "z"
    },
    "attitude": {
      "table": "vehicle_attitude",
      "q0": "q[0]",
      "q1": "q[1]",
      "q2": "q[2]",
      "q3": "q[3]"
    }
  }

## Sending data over the socket

In [7]:
send_zipped_pickle(socket, [datadict, [uav, uav_setpoint]])